# Projekt - report to pdf

# Import knihoven

In [77]:
import psycopg2
import openpyxl as opxl

import pandas as pd
import plotly.express as px
import regex as re

from pprint import pprint

# Import Datasetů 
- https://policie.gov.cz/clanek/statistika-nehodovosti.aspx
- Data z let 2023 - 2025
- Převedla jsem na formát xlsx
- Importováno do PostgreSQL (PostgreSQL database_dopravní nehody ČR)
- upraveny datumy a čas
- vytvořeny náhledy pro využití v další analýze

  # Připojení k databázi PostgreSQL

In [2]:
def execute_sql(sql_query: str) -> list: 
    connection = None
    data = None
    try:
        connection =  psycopg2.connect(
            host='localhost',
            user='postgres',
            password='kjm57',
            dbname='postgres')
    
        cursor = connection.cursor()
        cursor.execute(sql_query)
        data = cursor.fetchall()
        colnames = [cell[0] for cell in cursor.description]
        df_name = pd.DataFrame(data, columns=colnames)
    except Exception as E:
        print('ERROR')
    
    finally:
        if connection is not None:
            connection.close()

        return df_name

# Úprava a čištění dat

In [3]:
df_animal_involvement = execute_sql("SELECT * FROM dopravni_nehody_cr.animal_involvement")
df_accidents_in_time = execute_sql("SELECT * FROM dopravni_nehody_cr.accidents_in_time")
df_drivers = execute_sql("SELECT * FROM dopravni_nehody_cr.drivers")
df_first_aid = execute_sql("SELECT * FROM dopravni_nehody_cr.first_aid")
df_pedestrian_involvement = execute_sql("SELECT * FROM dopravni_nehody_cr.pedestrian_involvement")
df_accidents_crash = execute_sql("SELECT * FROM dopravni_nehody_cr.accidents_crash")
df_column_names = execute_sql("SELECT * FROM dopravni_nehody_cr.column_names")
df_data_description = execute_sql("SELECT * FROM dopravni_nehody_cr.data_description")

 ## Vytvoření slovníku pro párování dat

In [4]:
df_accidents_in_time.groupby('accident_year')['p1'].nunique().max()

np.int64(94945)

In [5]:
# Funkce pro kontrolu prázdných hodnot
def null_values(table):
    counts = table.isna().sum()
    df_na_check = pd.DataFrame({
        'column': counts.index,
        'na_count': counts.values,
        'na%': (counts.values / len(table) * 100).round(2)
    })
    df_na_check = df_na_check.merge(df_column_names,
                                    left_on='column',
                                    right_on='code',
                                    how='left')
    df_na_check = df_na_check.drop(columns=['code','table_name'])
    df_na_check = df_na_check[['column','descr','name_column_en','na_count','na%']]

    row_count = pd.DataFrame({
        'column': ['TOTAL'],
        'descr': ['Celkový počet záznamů'],
        'name_column_en': ['Total row count'],
        'na_count': [len(table)],
        'na%': [' ']
    })
    df_na_check = pd.concat([df_na_check, row_count], ignore_index=True)
    return df_na_check

def get_table_column_names(table):
    column_names = table.columns
    df_column_info = pd.DataFrame({
        'column': column_names
    })
    df_column_info = df_column_info.merge(df_column_names,
                                          left_on='column',
                                          right_on='code',
                                          how='left')
    df_column_info = df_column_info.drop(columns=['code'])
    return df_column_info


def ratio_in_category(data, groupbys, counted_cat_ratio):
    table_group = data.groupby(groupbys).size().reset_index(name='total')
    seskupení = data.groupby([groupbys, counted_cat_ratio])[counted_cat_ratio].size().reset_index(name='total_cat')
    table = pd.merge(seskupení, 
                     table_group, 
                     on=groupbys, 
                     how='left')
    table['ratio'] = table['total_cat']/table['total']*100
    return table

# Překladač sloupců a kategorií daných sloupců
def get_table_column_name(column):
    column_name = execute_sql(f"""SELECT descr FROM dopravni_nehody_cr.column_names
                              WHERE code = '{column}'""")
    return column_name.iloc[0, 0]

def categories_translate(table, column):
    cat_items = execute_sql(f"""SELECT id_detail, description_detail_2 FROM dopravni_nehody_cr.data_description
                                   WHERE column_code = '{column}'""")
    if cat_items is None or cat_items.empty:
        return table
    df = pd.merge(table,
                  cat_items,
                  left_on=column,
                  right_on='id_detail',
                  how='left')
    rename_column = get_table_column_name(column)
    df = df.rename(columns={'description_detail_2': rename_column})
    df = df.drop(columns=['id_detail', column])
    return df

def translate(table):
    varchars = table.select_dtypes(include=['object']).columns

    for column in varchars:
        table = categories_translate(table, column)
    return table

In [6]:
translate(df_accidents_crash)

,p1,id_vozidla,p2a,accident_year,accident_month,p4a,p4b,p13a,p13b,p13c,...,druh_pneumatik_na_vozidle,charakteristika_vozidla_,smyk,vozidlo_po_nehodě,způsob_vyproštění_osob_z_vozidla,kategorie_řidiče,stav_řidiče,vnější_ovlivnění_řidiče,označení_osoby,následky
0,25000028,1,02.01.2025,2025,1,0,14,0,0,0,...,zimní,soukromé nevyužívané k výdělečné činnosti,ne,nedošlo k požáru,nebylo třeba užít násilí,s řidičským oprávněním skupiny B,dobrý žádné nepříznivé okolnosti nebyly zjištěny,řidič nebyl ovlivněn,řidič,bez zranění
1,25000028,1,02.01.2025,2025,1,0,14,0,0,0,...,zimní,soukromé nevyužívané k výdělečné činnosti,ne,nedošlo k požáru,nebylo třeba užít násilí,s řidičským oprávněním skupiny B,dobrý žádné nepříznivé okolnosti nebyly zjištěny,řidič nebyl ovlivněn,řidič,bez zranění
2,25000031,2,02.01.2025,2025,1,0,13,0,0,0,...,zimní,"soukromá organizace (podnikatel, s.r.o., v.o.s...",ne,nedošlo k požáru,nebylo třeba užít násilí,s řidičským oprávněním skupiny B,dobrý žádné nepříznivé okolnosti nebyly zjištěny,řidič nebyl ovlivněn,řidič,bez zranění
3,25000031,2,02.01.2025,2025,1,0,13,0,0,0,...,zimní,"soukromá organizace (podnikatel, s.r.o., v.o.s...",ne,nedošlo k požáru,nebylo třeba užít násilí,s řidičským oprávněním skupiny B,dobrý žádné nepříznivé okolnosti nebyly zjištěny,řidič nebyl ovlivněn,řidič,bez zranění
4,25000031,2,02.01.2025,2025,1,0,13,0,0,0,...,zimní,"soukromá organizace (podnikatel, s.r.o., v.o.s...",ne,nedošlo k požáru,nebylo třeba užít násilí,s řidičským oprávněním skupiny B,dobrý žádné nepříznivé okolnosti nebyly zjištěny,řidič nebyl ovlivněn,spolucestující na předním sedadle vedle řidiče...,bez zranění
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
788328,70425000219,3,01.04.2025,2025,4,7,4,0,0,0,...,zimní,nezjištěno,ne,žádná z uvedených,nebylo třeba užít násilí,NaN,NaN,NaN,NaN,NaN
788329,151123000431,2,30.05.2023,2023,5,15,11,0,0,0,...,nezjištěno,"soukromá organizace (podnikatel, s.r.o., v.o.s...",ne,nedošlo k požáru,nebylo třeba užít násilí,NaN,NaN,NaN,NaN,NaN
788330,151123000431,2,30.05.2023,2023,5,15,11,0,0,0,...,nezjištěno,"soukromá organizace (podnikatel, s.r.o., v.o.s...",ne,nedošlo k požáru,nebylo třeba užít násilí,NaN,NaN,NaN,NaN,NaN
788331,24007430,2,16.08.2024,2024,8,0,13,0,0,0,...,letní,"soukromá organizace (podnikatel, s.r.o., v.o.s...",ne,žádná z uvedených,nebylo třeba užít násilí,NaN,NaN,NaN,NaN,NaN


In [75]:
translate(df_accidents_crash).columns

Index(['p1', 'p2a', 'accident_year', 'accident_month', 'p4a', 'p4b', 'd', 'e',
       'h', 'k', 'p13a', 'p13b', 'p13c', 'p14', 'p34', 'lokalita_nehody',
       'druh_nehody', 'druh_pevné_překážky', 'charakter_nehody',
       'zavinění_nehody', 'alkohol_u_viníka_nehody_přítomen',
       'drogy_u_viníka_nehody', 'hlavní_příčiny_nehody',
       'druh_povrchu_vozovky', 'stav_povrchu_vozovky_v_době_nehody',
       'stav_komunikace', 'povětrnostní_podmínky_v_době_nehody', 'viditelnost',
       'rozhledové_poměry', 'dělení_komunikace',
       'situování_nehody_na_komunikaci', 'místní_úprava_přednosti_v_jízdě',
       'specifická_místa_a_objekty_v_místě_nehody', 'směrové_poměry',
       'pohlaví_zraněné_osoby', 'následky_na_zraněném',
       'druh_pozemní_komunikace', 'druh_vozidla',
       'výrobní_značka_motorového_vozidla', 'druh_pohonu/paliva',
       'druh_pneumatik_na_vozidle', 'charakteristika_vozidla_', 'smyk',
       'vozidlo_po_nehodě', 'způsob_vyproštění_osob_z_vozidla',
       'kat

In [47]:
get_table_column_name('p29')

,descr
0,kategorie_chodce


In [91]:
df_but3 = execute_sql("""SELECT p1, accident_year, accident_month, p5a, p6, p8, p8a, p9, p10, p11, p11a, p12, p13a, p13b, p13c, p29, p29a, p30a, p30b, p33c, p33g, p34, id_vozidla, p44, p45a
                      FROM dopravni_nehody_cr.accidents_crash""")
df_but3 = translate(df_but3)
causes = sorted(df_but3['zavinění_nehody'].unique())

In [92]:
df_but3.columns

Index(['p1', 'accident_year', 'accident_month', 'p13a', 'p13b', 'p13c', 'p34',
       'id_vozidla', 'lokalita_nehody', 'druh_nehody', 'druh_pevné_překážky',
       'druh_zvěře/zvířete', 'charakter_nehody', 'zavinění_nehody',
       'alkohol_u_viníka_nehody_přítomen', 'drogy_u_viníka_nehody',
       'hlavní_příčiny_nehody', 'kategorie_chodce', 'reflexní_prvky_u_chodce',
       'alkohol_u_chodce_přítomen', 'druh_drogy_u_chodce',
       'pohlaví_zraněné_osoby', 'následky_na_zraněném', 'druh_vozidla',
       'výrobní_značka_motorového_vozidla'],
      dtype='object')

In [ ]:
df_but3[df_but3['druh_nehody']=='srážka s domácím zvířetem'].reset_index()

,index,p1,accident_year,accident_month,p13a,p13b,p13c,p34,id_vozidla,lokalita_nehody,...,drogy_u_viníka_nehody,hlavní_příčiny_nehody,kategorie_chodce,reflexní_prvky_u_chodce,alkohol_u_chodce_přítomen,druh_drogy_u_chodce,pohlaví_zraněné_osoby,následky_na_zraněném,druh_vozidla,výrobní_značka_motorového_vozidla
0,163,25000426,2025,1,0,0,0,1,2,v obci,...,nezjišťováno,nezaviněná řidičem,NaN,NaN,NaN,NaN,NaN,NaN,osobní automobil bez přívěsu,ŠKODA
1,164,25000426,2025,1,0,0,0,1,2,v obci,...,nezjišťováno,nezaviněná řidičem,NaN,NaN,NaN,NaN,NaN,NaN,osobní automobil bez přívěsu,ŠKODA
2,165,25000426,2025,1,0,0,0,1,2,v obci,...,nezjišťováno,nezaviněná řidičem,NaN,NaN,NaN,NaN,NaN,NaN,osobní automobil bez přívěsu,ŠKODA
3,166,25000426,2025,1,0,0,0,1,2,v obci,...,nezjišťováno,nezaviněná řidičem,NaN,NaN,NaN,NaN,NaN,NaN,osobní automobil bez přívěsu,ŠKODA
4,167,25000426,2025,1,0,0,0,1,2,v obci,...,nezjišťováno,nezaviněná řidičem,NaN,NaN,NaN,NaN,NaN,NaN,osobní automobil bez přívěsu,ŠKODA
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2131,770097,151523000732,2023,9,0,0,0,1,1,v obci,...,ne,nezaviněná řidičem,NaN,NaN,NaN,NaN,NaN,NaN,osobní automobil bez přívěsu,AUDI
2132,771649,170323000659,2023,11,0,0,0,1,1,mimo obec,...,nezjišťováno,nezaviněná řidičem,NaN,NaN,NaN,NaN,NaN,NaN,autobus,SOR
2133,771947,170623000863,2023,7,0,0,0,1,2,mimo obec,...,nezjišťováno,nezaviněná řidičem,NaN,NaN,NaN,NaN,NaN,NaN,osobní automobil bez přívěsu,KIA
2134,774018,180523001343,2023,9,0,0,0,1,1,v obci,...,ne,nezaviněná řidičem,NaN,NaN,NaN,NaN,NaN,NaN,osobní automobil bez přívěsu,ŠKODA


In [45]:
cause_conseqences = df_but3.groupby(['p1','zavinění_nehody','charakter_nehody']).size().reset_index()
#cause_conseqences = cause_conseqences.groupby(['zavinění_nehody','charakter_nehody'])['charakter_nehody'].size().reset_index(name='Počet_výskytů')
cause_conseqences = ratio_in_category(cause_conseqences, 'charakter_nehody', 'zavinění_nehody')
cause_conseqences

,zavinění_nehody,Pocet_vyskytu,ratio
0,chodcem,2651,1.00
1,jiné zavinění,1334,0.50
2,jiným účastníkem silničního provozu,562,0.21
3,"lesní zvěří, domácím zvířectvem",23145,8.70
4,technickou závadou vozidla,360,0.14
5,závadou komunikace,863,0.32
6,řidičem motorového vozidla,226267,85.09
7,řidičem nemotorového vozidla,10727,4.03


In [15]:
def top_3_in_cat(table, id_column, category_column):
    df_unique = table.drop_duplicates(subset=[id_column, category_column])
    top_3 = df_unique.groupby(category_column).size().reset_index(name='Pocet_vyskytu')
    top_3 = top_3.sort_values(by='Pocet_vyskytu', ascending=False)
    return top_3.head(3).reset_index()
def top_1_in_cat(table, id_column, category_column):
    return top_3_in_cat(table, id_column, category_column).head(1).reset_index()

In [20]:
top_3_in_cat(df_but3, 'p1', 'druh_nehody')
total = top_3_in_cat(df_but3, 'p1', 'druh_nehody')['Pocet_vyskytu'].sum()
print(total)

206776


In [ ]:
def ratio_in_category(data, id_column, counted_cat_ratio):
    df_unique = data.drop_duplicates(subset=[id_column, counted_cat_ratio])
    df_grouped = df_unique.groupby(counted_cat_ratio).size().reset_index(name='Pocet_vyskytu')
    total = df_grouped['Pocet_vyskytu'].sum()
    df_grouped['total'] = total
    df_grouped['ratio'] = (df_grouped['Pocet_vyskytu']/df_grouped['total']*100).round(2)
    df_grouped = df_grouped.drop(columns=['total'])
    df_grouped['Pocet_vyskytu'] = pd.to_numeric(df_grouped['Pocet_vyskytu'])
    df_grouped = df_grouped.sort_values('ratio', ascending=False)
    return df_grouped

In [ ]:
def category_conseq(table, filtered_value, category_col, consequences, graph_type):
    cause_conseqences = table.groupby(['p1',category_col,consequences]).size().reset_index()
    cause_conseqences = cause_conseqences.groupby([category_col,consequences])[consequences].size().reset_index(name='Pocet_vyskytu')
    filtered_cause_conseqences = cause_conseqences[cause_conseqences[category_col] == filtered_value].copy()
    filtered_cause_conseqences['total'] =  filtered_cause_conseqences['Pocet_vyskytu'].sum()
    filtered_cause_conseqences['ratio'] = (filtered_cause_conseqences['Pocet_vyskytu']/filtered_cause_conseqences['total']*100).round(2)
    filtered_cause_conseqences = filtered_cause_conseqences.drop(columns=['total'])
    filtered_cause_conseqences = filtered_cause_conseqences.sort_values('ratio', ascending=False)
    clean_label = consequences.replace('_', ' ')
    if graph_type == 'bar':
        filtered_cause_conseqences_graph = px.bar(filtered_cause_conseqences,
                                                  x=consequences,
                                                  y='Pocet_vyskytu',
                                                  color = 'Pocet_vyskytu',
                                                  color_continuous_scale='Reds',
                                                  text='Pocet_vyskytu',
                                                  title=f'{filtered_value} - {consequences}',
                                                  labels={'Pocet_vyskytu': 'Počet výskytů', consequences:clean_label})
    elif graph_type == 'pie': 
        filtered_cause_conseqences_graph = px.pie(filtered_cause_conseqences,
                                                  values='ratio',
                                                  names=consequences,
                                                  title=f'Poměr {consequences.replace("_", " ")} v {filtered_value}',
                                                  color_discrete_sequence=px.colors.sequential.Reds_r)
        filtered_cause_conseqences_graph.update_traces(marker=dict(line=dict(color='#000000', width=1)))
    unify_graphs(filtered_cause_conseqences_graph)
    return filtered_cause_conseqences_graph
    

In [ ]:
import requests

url = "https://meteostat.p.rapidapi.com/point/monthly"

querystring = {"lat":"49.8175","lon":"15.4730","alt":"43","start":"2020-01-01","end":"2020-12-31"}

headers = {
	"x-rapidapi-key": "6707bcbe21msh5a4bf753d11c559p19913cjsnbc6ca2acb548",
	"x-rapidapi-host": "meteostat.p.rapidapi.com"
}

response = requests.get(url, headers=headers, params=querystring)


if response.status_code == 200:
    data = response.json()  # Převede JSON na Python objekt
    response = requests.get(url, params=params, headers=headers)
    print("Data úspěšně stažena")
else:
    print(f"Chyba: {response.status_code}")

In [81]:
category_conseq(df_but3, 'chodcem', 'zavinění_nehody', 'charakter_nehody','pie')

In [93]:
category_conseq(df_but3, 'chodcem', 'zavinění_nehody', 'kategorie_chodce', 'bar')

In [78]:
count_couses = df_but3.groupby(['p1', 'zavinění_nehody'])['zavinění_nehody'].size().reset_index()
df_count_couses = count_couses.groupby('zavinění_nehody').size().reset_index(name='Počet_nehod')
count_couses_graph = px.barh(x='Počet_nehod',
                             y='zavinění_nehody',
                             orientation='h')
count_couses_graph.show()

ValueError: cannot insert zavinění_nehody, already exists

In [ ]:
# Překladač sloupců a kategorií daných sloupců
def get_table_column_name(column):
    column_name = execute_sql(f"""SELECT descr FROM dopravni_nehody_cr.column_names
                              WHERE code = '{column}'""")
    return column_name.iloc[0, 0]

def categories_translate(table, column):
    cat_items = execute_sql(f"""SELECT id_detail, description_detail_2 FROM dopravni_nehody_cr.data_description
                                   WHERE column_code = '{column}'""")
    if cat_items is None or cat_items.empty:
        return table
    df = pd.merge(table,
                  cat_items,
                  left_on=column,
                  right_on='id_detail',
                  how='left')
    rename_column = get_table_column_name(column)
    df = df.rename(columns={'description_detail_2': rename_column})
    df = df.drop(columns=['id_detail', column])
    return df

def translate(table):
    varchars = table.select_dtypes(include=['object']).columns

    for column in varchars:
        table = categories_translate(df_pedestrian_involvement, column)
    return table

In [65]:
df_pedestrian_involvement

,p1,p2a,accident_year,accident_month,p2b,d,e,p33d,p13a,p13b,...,pohlaví_zraněné_osoby,poskytnutí_první_pomoci_zraněnému,následky_na_zraněném,zavinění_nehody,alkohol_u_viníka_nehody_přítomen,drogy_u_viníka_nehody,hlavní_příčiny_nehody,stav_komunikace,povětrnostní_podmínky_v_době_nehody,specifická_místa_a_objekty_v_místě_nehody
0,70325000013,03.01.2025,2025,1,13:05,-1116137,-451262,23,0,1,...,muž,vozidlem RZP,lehké zranění,řidičem motorového vozidla,ne,ne,nepřizpůsobení rychlosti stavu vozovky (náledí...,jiný (neuvedený) stav nebo závada komunikace,neztížené,žádné nebo žádné z uvedených
1,70425000071,28.01.2025,2025,1,06:60,-1116854,-483626,52,0,0,...,muž,"neybyla poskytnuta, ale bylo nutno poskytnout",lehké zranění,řidičem motorového vozidla,ne,ne,řidič se plně nevěnoval řízení vozidla,"dobrý, bez závad",neztížené,žádné nebo žádné z uvedených
2,25000219,09.01.2025,2025,1,19:50,-1050176,-742322,62,0,0,...,muž,vozidlem RZP,lehké zranění,řidičem motorového vozidla,ne,nezjišťováno,nedání přednosti chodci na vyznačeném přechodu,"dobrý, bez závad",neztížené,přechod pro chodce
3,25000132,07.01.2025,2025,1,07:40,-1050020,-741447,15,0,0,...,žena,vozidlem RZP,lehké zranění,řidičem motorového vozidla,ne,ne,řidič se plně nevěnoval řízení vozidla,"dobrý, bez závad",neztížené,žádné nebo žádné z uvedených
4,25000098,05.01.2025,2025,1,11:30,-1051383,-741725,16,0,1,...,žena,"neybyla poskytnuta, ale bylo nutno poskytnout",lehké zranění,řidičem motorového vozidla,nezjišťováno,nezjišťováno,nepřizpůsobení rychlosti stavu vozovky (náledí...,"dobrý, bez závad",neztížené,přechod pro chodce
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9481,60225000036,13.01.2025,2025,1,11:30,-1161035,-598598,46,0,1,...,muž,vozidlem RZP,těžké zranění,řidičem motorového vozidla,"ano - obsah alkoholu v krvi 1,5 ‰ a více",ne,jiný druh nesprávného způsobu jízdy,"dobrý, bez závad",neztížené,žádné nebo žádné z uvedených
9482,180425000032,11.01.2025,2025,1,18:60,-990651,-674555,49,0,0,...,žena,vozidlem RZP,lehké zranění,řidičem motorového vozidla,ne,ne,nepřizpůsobení rychlosti stavu vozovky (náledí...,"dobrý, bez závad","nárazový vítr (boční, vichřice apod.)",žádné nebo žádné z uvedených
9483,180425000032,11.01.2025,2025,1,18:60,-990651,-674555,14,0,0,...,žena,vozidlem RZP,lehké zranění,řidičem motorového vozidla,ne,ne,nepřizpůsobení rychlosti stavu vozovky (náledí...,"dobrý, bez závad","nárazový vítr (boční, vichřice apod.)",žádné nebo žádné z uvedených
9484,41025000048,15.01.2025,2025,1,05:40,-973714,-762735,60,0,0,...,žena,jinou osobou,lehké zranění,řidičem motorového vozidla,ne,ne,nedání přednosti chodci na vyznačeném přechodu,"dobrý, bez závad",neztížené,přechod pro chodce


In [56]:
query_gps = execute_sql("SELECT DISTINCT p1, d, e, k, p4a, p5a, p6, p9 FROM dopravni_nehody_cr.accidents_crash WHERE d < 0")

In [102]:
pomer_v_categorii(query_gps,'k','p9')

,k,p9,celkem_cat,celkem,ratio
0,dálnice,1,2077,12642,16.429362
1,dálnice,2,10565,12642,83.570638
2,komunikace místní,1,22675,136273,16.639393
3,komunikace místní,2,113598,136273,83.360607
4,"komunikace účelová - ostatní (parkoviště, odpo...",1,290,5901,4.914421
5,"komunikace účelová - ostatní (parkoviště, odpo...",2,5611,5901,95.085579
6,komunikace účelová - polní a lesní cesty atd.,1,299,748,39.973262
7,komunikace účelová - polní a lesní cesty atd.,2,449,748,60.026738
8,silnice 1.třídy,1,11541,33936,34.008133
9,silnice 1.třídy,2,22395,33936,65.991867


In [70]:
categories = sorted(query_gps['p9'].unique())
categories

['1', '2']

In [57]:
query_gps

,p1,d,e,k,p4a,p5a,p6,p9
0,10123000001,-1085261,-708394,silnice 2.třídy,1,2,1,2
1,10123000002,-1099430,-733019,silnice 1.třídy,1,2,9,1
2,10123000003,-740204,-1079447,silnice 2.třídy,1,2,5,2
3,10123000004,-733929,-1091843,silnice 1.třídy,1,2,5,2
4,10123000005,-1089389,-732822,silnice 1.třídy,1,2,0,1
...,...,...,...,...,...,...,...,...
265864,74325000040,-1121296,-474585,komunikace místní,7,1,2,2
265865,74325000041,-1120590,-461226,dálnice,7,2,0,2
265866,74325000042,-1118433,-454075,dálnice,7,2,3,1
265867,74325000043,-1119269,-454924,dálnice,7,2,1,2


In [48]:
df_but2 = execute_sql(("""SELECT p1, d as lat, e as lon, k FROM dopravni_nehody_cr.gps"""))
df_but2[df_but2['lat'] > 0]

,p1,lat,lon,k


In [9]:
df_accidents_count = df_accidents_in_time.groupby(['Year', 'Month']).size().reset_index(name='Pocet')


In [10]:
fig = px.histogram(df_accidents_in_time, 
                   x="Month", 
                   facet_col="Year", 
                   facet_col_wrap=3) # Po 3 grafech na řádek

fig.update_layout(bargap=0.1)
fig.show()